In [13]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
import ignite

In [14]:
device = torch . device ( "cuda:0" if torch . cuda . is_available () else "cpu" )

In [15]:
from torch.utils.data import Dataset
import os

In [16]:
from natsort import natsorted
from PIL import Image

class TrainDataset(Dataset):
    def __init__(self, m_dir, transform):
        
        # Set the loading directory
        self.m_dir = m_dir
        self.transform = transform
        
        downs_imgs = os.listdir(os.path.join(m_dir, 'downs'))
        peaks_imgs = os.listdir(os.path.join(m_dir, 'peaks'))
        rises_imgs = os.listdir(os.path.join(m_dir, 'rises'))
        
        dimg = natsorted(downs_imgs)
        pimg = natsorted(peaks_imgs)
        rimg = natsorted(rises_imgs)
        self.total_imgs = dimg + pimg + rimg
        
    
    def __len__(self):
        # Return the previously computed number of images
        return len(self.total_imgs)
    
    
    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
     
        # Use PIL for image loading
        image = Image.open(img_loc).convert("RGB")
        # Apply the transformations
        tensor_image = self.transform(image)
        return tensor_image

    

In [17]:
dataset = TrainDataset(m_dir=".\\dataset\\Train", transform=transform)
dataloader = DataLoader(dataset)
print(next(iter(dataloader)).shape)  # prints shape of image with single batch

NameError: name 'transform' is not defined


    '''
    trainset_folder = 'dataset/Train'
        
    for downs in os.listdir(trainset_folder):
        down_folder = os.path.join(trainset_folder, 'downs')
        with open(down_folder, 'r') as downs_file:
            for name in downs_file.read().splitlines():
                self.samples.append((type))
            
    for peaks in os.listdir(trainset_folder):
        peaks_folder = os.path.join(trainset_folder, 'peaks')
        with open(peaks_folder, 'r') as peaks_file:
            for name in peaks_file.read().splitlines():
                self.samples.append((type))
            
    for rises in os.listdir(trainset_folder):
        rises_folder = os.path.join(trainset_folder, 'rises')
        with open(rises_folder, 'r') as rises_file:
            for name in rises_file.read().splitlines():
                self.samples.append((type))
            


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


if __name__ == '__main__':
    dataset = TrainDataset('dataset/Train')
    print(len(dataset))
    print(dataset[420])